In [1]:
#imports
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
import seaborn as sns

from PathFinder import find_path

In [2]:
#read data
PATH = find_path()

#select files
#files = ['application_train.csv','application_test.csv',]
files = ['preprocessed_train.csv', 'preprocessed_test.csv']
"""         
          'bureau.csv', 
         'bureau_balance.csv', 'credit_card_balance.csv',
         'installments_payments.csv', 'POS_CASH_balance.csv',
         'previous_application.csv']
"""
data_files=[]
data=[]
print("files used: {}".format(files))

#actually read files
for file in files:
    stuff = pd.read_csv(PATH+file,header=0,engine='python')
    #print(stuff)
    data_files.append(file)
    data.append(stuff)

files used: ['preprocessed_train.csv', 'preprocessed_test.csv']


In [3]:
#select and convert columns
original_train = data[0] #save the original because we'll need some stuff later
original_test = data[1]
app_train = data[0].copy()
app_test = data[1].copy()

#delete ID cols
del app_train['SK_ID_CURR']
del app_test['SK_ID_CURR']

app_train_y = app_train['TARGET']
new_cols_x = app_train.columns.values.tolist()
new_cols_x.remove('TARGET')
app_train_x = app_train[new_cols_x]

print('Training Features shape: ', app_train_x.shape) #235 cols
print('Testing Features shape: ', app_test.shape) #235 cols

Training Features shape:  (307511, 235)
Testing Features shape:  (48744, 235)


In [4]:
#apply logistic regression
from sklearn.linear_model import LogisticRegression

# Make the model with the specified regularization parameter
log_reg = LogisticRegression(C = 0.0001, random_state=20)

# Train on the training data
log_reg.fit(app_train_x, app_train_y)

LogisticRegression(C=0.0001, class_weight=None, dual=False,
          fit_intercept=True, intercept_scaling=1, max_iter=100,
          multi_class='ovr', n_jobs=1, penalty='l2', random_state=20,
          solver='liblinear', tol=0.0001, verbose=0, warm_start=False)

In [5]:
# Make predictions
# Make sure to select the second column only
log_reg_pred = log_reg.predict_proba(app_test)[:, 1]

In [6]:
# Submission dataframe
submit = original_test[['SK_ID_CURR']]
submit['TARGET'] = log_reg_pred

submit.head()

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


,SK_ID_CURR,TARGET
0,100001,0.063382
1,100005,0.121217
2,100013,0.086837
3,100028,0.057044
4,100038,0.126813


In [7]:
# Save the submission to a csv file
overwrite = False
if overwrite:
    submit.to_csv('results/log_reg_baseline_1.csv', index = False)

Submitted to kaggle
score: 0.67825

In [11]:
print(log_reg.coef_.shape)

(1, 235)


In [26]:
print('intercept:', log_reg.intercept_[:,None][0][0])
print('weights:',log_reg.coef_[0])
#np.hstack((log_reg.intercept_[:,None], log_reg.coef_))
#print('weights:')
#for feature, weight in zip(new_cols_x, log_reg.coef_[0]):
#    #print('{:3.7} {}'.format(weight, feature))
#    print(f'{weight} {feature}')

intercept: -0.1446161740261535
weights: [ 7.27818824e-04 -1.86423107e-04 -3.32199722e-02 -1.50719384e-02
 -3.64293571e-02 -7.23428046e-02  2.35608396e-02 -8.16525188e-02
 -7.61791017e-02 -1.38509605e-02 -1.02948424e-02 -1.44607106e-01
 -6.28656670e-02  2.87920429e-02 -1.43773765e-01 -7.19515502e-02
 -7.42820867e-03 -5.42853070e-03  9.69945861e-03  1.21166228e-02
 -9.28765341e-02 -7.22125915e-04 -5.71404678e-03 -7.68651364e-03
  4.16954933e-02  3.34111779e-02  2.92819425e-03 -1.59049112e-01
 -3.18926796e-01 -2.98898814e-01 -1.81823678e-02 -1.37620109e-02
 -1.40907383e-01 -1.09272687e-01 -4.83642060e-03 -1.16704759e-02
 -2.38942107e-02 -3.85653182e-02 -3.51980423e-02 -8.43953502e-03
 -1.29160508e-02 -1.77327599e-02 -3.16420861e-04 -3.87118010e-03
 -1.71863404e-02 -1.31427557e-02 -1.40731089e-01 -1.10570472e-01
 -4.28160224e-03 -1.06075825e-02 -2.32357995e-02 -3.79532031e-02
 -3.48500235e-02 -8.10251166e-03 -1.31410016e-02 -1.70561413e-02
 -1.46718922e-04 -3.61870782e-03 -1.80911432e-02 -